In [ ]:
## Data transformation from bronze to gold: 
## Bronze -> Silver: Changing the time formats to company used format 
from pyspark.sql.functions import from_utc_timestamp, date_format 
from pyspark.sql.types import TimestampType 
table_name = []

for i in dbutils.fs.ls('mnt/bronze/SalesLT/'):                             
    table_name.append(i.name.split('/')[0])



for i in table_name: 
    path = '/mnt/bronze/SalesLT/' + i + '/' + i + '.parquet'
    df = spark.read.format('parquet').load(path)
    column = df.columns
    for col in column:
        if "Date" in col or "date" in col: 
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), 'UTC'), 'yyyy-MM-dd'))    ## Change time format
    output_path = '/mnt/silver/SalesLT/' + i + '/' + 'tmp'
    df.write.format('parquet').mode('overwrite').save(output_path) 

    out_file_name = i + '.parquet'
    path2 = "mnt/silver/SalesLT/" + i + '/'
    tmp_path = "mnt/silver/SalesLT/" + i + '/' + 'tmp'
    file = dbutils.fs.ls(tmp_path)[-1][0]
    dbutils.fs.cp(file,path2+out_file_name)
    dbutils.fs.rm(tmp_path, True)

## Silver -> Gold: Changing columns names to company standards (lowercase + underscores for seperation)
table_name = []

for i in dbutils.fs.ls('mnt/silver/SalesLT/'):
    table_name.append(i.name.split('/')[0])
for name in table_name: 
    path = '/mnt/silver/SalesLT/' + name 
    df = spark.read.format('parquet').load(path)
    
    column_names = df.columns

    for old_col_name in column_names: 
        new_col_name = "".join(["_" + char if char.isupper() and not old_col_name[i - 1].isupper() else char for i, #Add underscore, change capitalization
                                char in enumerate(old_col_name)]).lstrip("_")   #Remove leading underscores
        df = df.withColumnRenamed(old_col_name, new_col_name)
        
    output_path = '/mnt/gold/SalesLT/' + name + '/' + 'tmp'
    df.write.format('parquet').mode('overwrite').save(output_path) 

    out_file_name = name + '.parquet'
    path2 = "mnt/gold/SalesLT/" + name + '/'
    tmp_path = "mnt/gold/SalesLT/" + name + '/' + 'tmp'
    file = dbutils.fs.ls(tmp_path)[-1][0]
    dbutils.fs.cp(file,path2+out_file_name)
    dbutils.fs.rm(tmp_path, True)
